# Automatic labelling of topics for United Nations General Debate Corpus (UNGDC) 

This program consists of two parts: First, the topics occurring in United Nations General Debate Corpus (UNGDC) are discovered. Second, the discovered topics are automatically labelled using the pipeline offered as part of the publication “Automatic labelling of topics using neural embeddings” by Bhatia et al in 2016.

In the first part, using Gensim library in Python, an LDA model is trained on the corpus and semantic coherence measure is used to find an optimal number of topics. The program outputs the optimal number of topics along with their 10 most frequent words.

In the second part, the discovered topic along with their 10 most frequent words is fed to the pipeline in a .csv file. Each line in the file contains the topic id (here: 0 to 10) along with top n (here 10) words. The path to this file is updated in get_labels.py.

Finally, the get_labels.py should should be run by typing "python get_labels.py -cg –us -s ” in the command prompt. It will give you candidate labels in a file with unsupervised and supervised labels printed on the console as well as in output files. By default output files namely output_candidates, output_unsupervised, and output_supervised will be created in the same directory.

The only difference between this program and its earlier version is that in the first program, we use single words to topic model with LDA, but in this program we use bigrams and trigrams as well.

As you will see in the end, the algorithm is not able to produce labels for topic 4.

# Import the required packages

In [5]:
#Taken From:https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/
from pprint import pprint
import numpy as np
import os,re
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity

from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
from numpy import array

# Import the dataset


In [6]:
docs=[]
pattern='^[A-Z]{3}_[0-9]{2}_[0-9]{4}.txt$'
path='D:/UN/UNGDC_1970-2017/Converted sessions/'
for (dirname, dirs, files) in os.walk(path):
   for filename in files:
        thefile = os.path.join(dirname,filename)
        with open (thefile,encoding='utf8',errors='ignore') as fin:
             if re.match(pattern, filename):

                doc=fin.read().strip('\n\t')
                docs.append(doc)       
print(len(docs))

7871


# Define a function for tokenizing and lemmatizing

In [7]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs
# Perform function on our document
docs = docs_preprocessor(docs)


# Create Biagram & Trigram Models 

In [8]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs,minimum count 10 means only that appear 10 times or more.
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a trigram, add to document.
            docs[idx].append(token)


# Remove rare & common tokens 

In [9]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 41184
Number of documents: 7871


# Build The Model

In [10]:
# Set parameters.
num_topics = 5
chunksize = 500 
passes = 20 
iterations = 400
eval_every = 1  

# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 5 topics
print(lda_model.print_topics())

[(0, '0.002*"latin_american" + 0.002*"central_america" + 0.002*"united_kingdom" + 0.002*"central_american" + 0.002*"young_people" + 0.002*"natural_resource" + 0.002*"this_rostrum" + 0.002*"costa_rica" + 0.002*"rather_than" + 0.001*"this_hall"'), (1, '0.006*"african_union" + 0.006*"democratic_republic" + 0.005*"central_african" + 0.005*"least_developed" + 0.003*"caribbean" + 0.003*"sierra_leone" + 0.003*"guinea_bissau" + 0.003*"african_continent" + 0.002*"guinea" + 0.002*"western_sahara"'), (2, '0.008*"republic_korea" + 0.008*"soviet_union" + 0.005*"korean_peninsula" + 0.003*"korean" + 0.003*"foreign_policy" + 0.003*"south_east" + 0.003*"republic_moldova" + 0.003*"central_asia" + 0.003*"east_west" + 0.003*"peace_keeping"'), (3, '0.003*"third_world" + 0.003*"south_african" + 0.002*"arab_emirate" + 0.002*"islamic" + 0.002*"islamic_republic" + 0.002*"right_self" + 0.002*"cyprus" + 0.002*"legitimate_right" + 0.002*"yemen" + 0.002*"just_lasting"'), (4, '0.005*"european_union" + 0.003*"small_

# Find the optimal number of topics

In [11]:
#Using c_v Measure
def compute_coherence_values(dictionary, corpus, texts, limit, start=10, step=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


# Create a model list and plot Coherence score against a number of topics

In [12]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=docs, start=10, limit=30, step=5)
# Show graph
import matplotlib.pyplot as plt
limit=30; start=10; step=5;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [13]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 10  has Coherence Value of 0.423
Num Topics = 15  has Coherence Value of 0.3896
Num Topics = 20  has Coherence Value of 0.3752
Num Topics = 25  has Coherence Value of 0.4056


# Selecting The Optimal Model

In [14]:
#finding the number of topics that gives the highest Coherence Value 
max_cv=max(coherence_values)
#finding the index of the model with this numnber of topics
index_model=coherence_values.index(max_cv)
#print(index_model)
# Select the model and print the topics 
optimal_model = model_list[index_model]
model_topics = optimal_model.show_topics(formatted=False)
#pprint(optimal_model.print_topics(num_words=10))

# Writing the topics along with the 10 most frequent words in a file

In [15]:
import re
with open('D:/UN/Topics_UN_B.csv','w') as fw:
    fw.write("topic_id,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9"+"\n")
    for i in optimal_model.print_topics():
        r1 = re.findall(r"\w+",i[1])
        line=str(i[0])
        for item in r1:
            if not item.isdigit():
                line=line+','+item
        fw.write(line+'\n')

# Displaying the file

In [16]:
from IPython.display import display
import pandas
df = pandas.read_csv("D:/UN/Topics_UN_B.csv")
display(df)

,topic_id,term0,term1,term2,term3,term4,term5,term6,term7,term8,term9
0,0,trinidad_tobago,peace_keeping,least_developed,trinidad,tobago,african_union,third_world,european_union,somalia,peaceful_settlement
1,1,small_island,solomon_island,millennium_development,caribbean,least_developed,developing_state,natural_disaster,saint_lucia,most_vulnerable,african_union
2,2,pacific,republic_korea,japan,pacific_island,papua_guinea,small_island,burkina_faso,north_korea,millennium_development,timor_leste
3,3,republic_korea,central_asia,republic_moldova,korean_peninsula,equatorial_guinea,guinea,european_union,guinea_bissau,foreign_policy,kosovo
4,4,european_union,african_union,good_governance,around_world,millennium_development,twenty_first,bosnia_herzegovina,armed_conflict,civil_society,peacekeeping_operation
5,5,central_america,sierra_leone,latin_american,costa_rica,central_american,drug_trafficking,democratic_republic,cuba,young_people,natural_resource
6,6,guinea_bissau,guinea,democratic_republic,third_world,papua_guinea,south_african,chad,bissau,western_sahara,comoros
7,7,syria,chemical_weapon,islamic_republic,islamic,antigua_barbuda,soviet_union,egypt,weapon_mass,foreign_policy,third_world
8,8,south_sudan,arab_emirate,syria,syrian,call_upon,pakistan,sudan,weapon_mass,yemen,islamic
9,9,central_african,democratic_republic,morocco,soviet_union,third_world,cyprus,african_union,kingdom_morocco,foreign_policy,take_into


# The file is fed to the paper's model and the unsupervised Labels are:

In [20]:
#The Labels are saved into a file named output_unsupervised23.txt
#The following code changes the file into CSV format to be shown here
with open("d:/UN/output_unsupervised23.txt") as fi:
    content=fi.read().strip().split("top 3 labels for topic")

with open ("d:/UN/Labels_un","w") as fw:
    fw.write("Topic_id,Label_1,Label_2,Label_3"+'\n')
    Topic_id=0
    for i in content:
       i=i.split('\n')
       if len(i)>3:         
         fw.write(str(Topic_id)+','+i[1]+','+i[2]+','+i[3]+'\n')
         Topic_id+=1
    
df = pandas.read_csv("d:/UN/Labels_un")
display(df)

,Topic_id,Label_1,Label_2,Label_3
0,0,dominica,jamaica,caribbean
1,1,turks_and_caicos_islands,saint_kitts_and_nevis,saint_kitts
2,2,marshall_islands,korea,okinawa_island
3,3,central_african_republic,liberia,mauritania
4,4,NaN,NaN,NaN
5,5,central_america,latin_america,south_america
6,6,guinea-bissau,equatorial_guinea,central_african_republic
7,7,islamic_state,islamism,islamic_fundamentalism
8,8,sudan,syrian_civil_war,yemen
9,9,central_african_republic,africa,tunisia


As you can see the paper's algorithm is not able to produce labels for topic 4.

# The file is fed to the paper's model and the supervised Labels are:

In [23]:
#The Labels are saved into a file named output_supervised23.txt
#The following code changes the file into CSV format to be shown here
with open("d:/UN/output_supervised23.txt") as fi:
    content=fi.read().strip().split("Top 3 labels for topic")

with open ("d:/UN/Labels_su","w") as fw:
    fw.write("Topic_id,Label_1,Label_2,Label_3"+'\n')
    Topic_id=0
    for i in content:
       i=i.split('\n')
       if len(i)>3:         
         fw.write(str(Topic_id)+','+i[1]+','+i[2]+','+i[3]+'\n')
         Topic_id+=1
    
df = pandas.read_csv("d:/UN/Labels_su")
display(df)

,Topic_id,Label_1,Label_2,Label_3
0,0,jamaica,dominica,caribbean
1,1,turks_and_caicos_islands,saint_kitts_and_nevis,guadeloupe
2,2,marshall_islands,korea,south_korea
3,3,sudan,liberia,sierra_leone
4,4,NaN,NaN,NaN
5,5,south_america,central_america,latin_america
6,6,equatorial_guinea,guinea-bissau,central_african_republic
7,7,egypt,syria,islamic_state
8,8,sudan,yemen,syrian_civil_war
9,9,africa,tunisia,ethiopia


As you can see the paper's algorithm is not able to produce labels for topic 4.